In [ ]:
import os 

from dotenv import find_dotenv, load_dotenv

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(model='text-curie-001', temperature=0.9)
text = 'What would be a good company name for a company selling donuts?'
output = llm(text)
print(output)

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
llm = HuggingFaceHub(
    repo_id='google/flan-t5-large', 
    model_kwargs={'temperature': 1, 'max_length': 64},
)
text = 'Translate English to German: How are you?'
output = llm(text)
print(output)

## Prompt Templates

In [ ]:
prompt = """Questions: Can Barack Obama have a conversation with George Washington?
Let's think step by step.
Answer:"""
llm(prompt)

In [ ]:
from langchain import PromptTemplate


In [ ]:
template = """Questions: {question}
Let's think step by step.
Answer:"""
prompt = PromptTemplate(template=template, input_variables=['question'])
print(prompt.format(question='Can Barack Obama have a conversation with George Washington?'))

## LLMChain

In [ ]:
from langchain import LLMChain

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)
question = 'Can Barack Obama have a conversation with George Washington?'
print(chain.run(question))

## Agents and Tools

In [ ]:
from langchain.agents import load_tools, initialize_agent, AgentType

In [ ]:
llm = OpenAI(model='text-davinci-003', temperature=0)
tools = load_tools(['wikipedia', 'llm-math'], llm=llm)

In [ ]:
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
)

In [ ]:
agent.run('What is the release year of the file Inception directed by Christopher Nolan? What the value of using this year to raise power of 2?')

## Memory

In [ ]:
from langchain import ConversationChain

In [ ]:
llm = OpenAI(model='text-davinci-003', temperature=0.9)
conversation = ConversationChain(llm=llm, verbose=True)

In [ ]:
conversation.predict(input='Hi there!')

In [ ]:
conversation.predict(input='Can we talk about AI?')

In [ ]:
conversation.predict(input='I like to learn about few-shot learning?')

## Retrieval -- Document loader

In [ ]:
!unzip ./data/notion-interview.zip -d ./data/Notion_DB

In [ ]:
from langchain.document_loaders import NotionDirectoryLoader

In [ ]:
loader = NotionDirectoryLoader(os.path.join('data', 'Notion_DB'))
docs = loader.load()
print(*docs, sep='\n')

## Vector Stores -- Faiss

Facebook AI Similarity Search (Faiss): a library for similarity search and clustering of dense vectors.

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
loader = TextLoader(os.path.join('data', 'state_of_the_union.txt'))
document = loader.load()

print(*document, sep='\n')

In [ ]:
# Split 1 document into multiple documents, with maximum length of 1000 tokens 
# per document:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(document)

In [ ]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

In [ ]:
docs_and_scores = db.similarity_search_with_score(query)
print(docs_and_scores[0])

In [ ]:
# Save and load FAISS index
path_faiss = os.path.join('outputs', 'faiss_index')
db.save_local(path_faiss)

new_db = FAISS.load_local(path_faiss, embeddings)
docs = new_db.similarity_search(query)
print(docs[0])